# Overview

The purpose of this project is to find the happiness and freedom of the populace and combine the data to see where the rankings are for the most “free people” and if that correlates to their happiness ranking. In this report, we'll describe the entire ETL (Extract, Transform, Load) process.

# Extract

### Store CSV into DataFrame

Using pandas, we stored our information from the `happiness.csv` and the `freedom.csv` into separate DataFrames.

In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine, inspect

In [2]:
# Read CSV files and load into dataframe
happiness_file = "Resources/happiness.csv"
happiness_df = pd.read_csv(happiness_file)
happiness_df.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.8087,0.031156,7.869766,7.747634,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,Western Europe,7.6456,0.033492,7.711245,7.579955,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,Western Europe,7.5599,0.035014,7.628528,7.491272,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,Western Europe,7.5045,0.059616,7.621347,7.387653,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,Western Europe,7.4880,0.034837,7.556281,7.419719,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266


In [3]:
# Read CSV files and load into dataframe
freedom_file = "Resources/freedom.csv"
freedom_df = pd.read_csv(freedom_file)
freedom_df.head()

,year,countries,ISO,region,hf_score,hf_rank,hf_quartile,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,...,ef_regulation_business_adm,ef_regulation_business_bureaucracy,ef_regulation_business_start,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank
0,2019,Albania,ALB,Eastern Europe,8.14,43.0,2.0,5.97,4.76,4.26,...,5.65,6.67,9.74,6.24,5.62,7.18,6.85,7.70,7.81,31.0
1,2019,Algeria,DZA,Middle East & North Africa,5.26,154.0,4.0,5.21,5.64,4.35,...,4.22,2.22,9.31,2.58,8.77,7.03,5.69,5.84,4.90,162.0
2,2019,Angola,AGO,Sub-Saharan Africa,6.09,129.0,4.0,2.72,4.43,3.60,...,2.94,2.44,8.73,4.70,7.92,6.78,5.59,5.97,5.50,153.0
3,2019,Argentina,ARG,Latin America & the Caribbean,7.38,74.0,2.0,6.83,5.94,4.35,...,2.71,5.78,9.58,6.53,5.73,6.51,6.14,5.99,5.50,153.0
4,2019,Armenia,ARM,Caucasus & Central Asia,8.20,40.0,1.0,NaN,NaN,NaN,...,5.17,5.56,9.86,6.96,9.30,7.04,7.32,7.82,8.03,15.0


# Transform

### Transform Happiness DataFrame

We checked the column names and filtered the original DataFrame to contain the following columns: Country name, Regional indicator, Ladder score, Social support, Freedom to make life choices, Healthy life expectancy, Logged GDP per capita. We renamed these columns, getting rid of unnecessary spaces and capital letters.

In [4]:
# Check columns
for col in happiness_df.columns:
    print(col)

Country name
Regional indicator
Ladder score
Standard error of ladder score
upperwhisker
lowerwhisker
Logged GDP per capita
Social support
Healthy life expectancy
Freedom to make life choices
Generosity
Perceptions of corruption
Ladder score in Dystopia
Explained by: Log GDP per capita
Explained by: Social support
Explained by: Healthy life expectancy
Explained by: Freedom to make life choices
Explained by: Generosity
Explained by: Perceptions of corruption
Dystopia + residual


In [5]:
# Create a filtered dataframe from specific columns
happiness_cols = ["Country name", "Regional indicator", "Ladder score", "Social support", "Freedom to make life choices", "Healthy life expectancy", "Logged GDP per capita"]
happiness_transformed = happiness_df[happiness_cols].copy()

# Rename the column headers
happiness_transformed = happiness_transformed.rename(columns = { "Country name": "country",
                                                               "Regional indicator": "region",
                                                               "Ladder score": "happiness_score",
                                                               "Social support": "social_support",
                                                               "Freedom to make life choices": "freedom_of_choice",
                                                               "Healthy life expectancy": "life_expectancy",
                                                               "Logged GDP per capita": "gdp"})

happiness_transformed.head()

,country,region,happiness_score,social_support,freedom_of_choice,life_expectancy,gdp
0,Finland,Western Europe,7.8087,0.954330,0.949172,71.900825,10.639267
1,Denmark,Western Europe,7.6456,0.955991,0.951444,72.402504,10.774001
2,Switzerland,Western Europe,7.5599,0.942847,0.921337,74.102448,10.979933
3,Iceland,Western Europe,7.5045,0.974670,0.948892,73.000000,10.772559
4,Norway,Western Europe,7.4880,0.952487,0.955750,73.200783,11.087804


### Transform Freedom DataFrame

We checked the column names and filtered the original DataFrame to contain only the year 2019 with the following columns: countries, region, hf_score, pf_ss_homicide, pf_expression_freedom, ef_government_consumption, pf_religion, pf_identity_same, "ef_money", pf_rol_civil, pf_rol_criminal. We gave the columns easier to understand names, getting rid of unnecessary spaces and capital letters. We decided not to drop null values as it would have eliminated too many rows. Many rows contain missing values but our identifying information, country and region, as well as our most important metric, human freedom score were all present. 

In [6]:
# Check columns
for col in freedom_df.columns:
    print(col)

year
countries
ISO
region
hf_score
hf_rank
hf_quartile
pf_rol_procedural
pf_rol_civil
pf_rol_criminal
pf_rol
pf_ss_homicide
pf_ss_disappearances_disap
pf_ss_disappearances_violent
pf_ss_disappearances_organized
pf_ss_disappearances_fatalities
pf_ss_disappearances_injuries
pf_ss_disappearances_torture
pf_ss_killings
pf_ss_disappearances
pf_ss
pf_movement_vdem_foreign
pf_movement_vdem_men
pf_movement_vdem_women
pf_movement_vdem
pf_movement_cld
pf_movement
pf_religion_suppression
pf_religion_freedom_vdem
pf_religion_freedom_cld
pf_religion_freedom
pf_religion
pf_assembly_entry
pf_assembly_freedom_house
pf_assembly_freedom_bti
pf_assembly_freedom_cld
pf_assembly_freedom
pf_assembly_parties_barriers
pf_assembly_parties_bans
pf_assembly_parties_auton
pf_assembly_parties
pf_assembly_civil
pf_assembly
pf_assembly_rank
pf_expression_killed
pf_expression_jailed
pf_expression_media
pf_expression_cultural
pf_expression_gov
pf_expression_internet
pf_expression_harass
pf_expression_selfcens
pf_expre

In [7]:
# Filter out all but 2019
freedom_2019 = freedom_df.loc[freedom_df["year"] == 2019]
freedom_2019.head()

# Create a filtered dataframe from specific columns
freedom_cols = ["countries", "region", "hf_score", "pf_ss_homicide", "pf_expression_freedom", "ef_government_consumption", "pf_religion", "pf_identity_same", "ef_money", "pf_rol_civil", "pf_rol_criminal"]
freedom_transformed = freedom_2019[freedom_cols].copy()

# Rename the column headers
freedom_transformed = freedom_transformed.rename(columns = { "countries": "country",
                                                               "region": "region",
                                                               "hf_score": "human_freedom_score",
                                                               "pf_ss_homicide": "homicide",
                                                               "pf_expression_freedom": "freedom_of_expression",
                                                               "ef_government_consumption": "government_consumption",
                                                               "pf_religion": "freedom_of_religion",
                                                               "pf_identity_same": "same_sex_relationships",
                                                               "ef_money": "freedom_of_money",
                                                               "pf_rol_civil": "civil_justice",
                                                               "pf_rol_criminal": "criminal_justice"})

freedom_transformed.head()

,country,region,human_freedom_score,homicide,freedom_of_expression,government_consumption,freedom_of_religion,same_sex_relationships,freedom_of_money,civil_justice,criminal_justice
0,Albania,Eastern Europe,8.14,9.33,6.00,8.16,9.74,10.0,9.86,4.76,4.26
1,Algeria,Middle East & North Africa,5.26,9.60,5.00,3.52,5.45,0.0,7.34,5.64,4.35
2,Angola,Sub-Saharan Africa,6.09,8.59,5.50,7.29,6.95,5.0,4.93,4.43,3.60
3,Argentina,Latin America & the Caribbean,7.38,8.46,8.25,5.99,9.86,10.0,4.00,5.94,4.35
4,Armenia,Caucasus & Central Asia,8.20,9.51,8.25,7.94,8.55,10.0,9.65,NaN,NaN


# Load

### Create SQL DataBase

First, we sketched out an Entity Relationship Diagram using QuickDBD. Then we created a DataBase, `etlproject_db`, in Postgresql as well as two tables, `world_happiness` and `human_freedom_index`, with the column names from our transformed DataFrame in the previous section.

![ERD](Images/ERD.png)

### Create DataBase Connection

We used SQLAlchemy's ``create_engine`` to connect to our newly created SQL DataBase. Using SQLAlchemy's `inspector` to retrieve the table names, we confirmed our connection to the DataBase.

In [8]:
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'etlproject_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [9]:
# Confirm Tables
inspector = inspect(engine)
inspector.get_table_names()

['regions', 'world_happiness', 'human_freedom_index']

### Load DataFrames
We loaded the above dataframes, `happiness_transformed` and `freedom_transformed`, into our SQL databases, `world_happiness` and `human_freedom_index` respectively, and left off the indices. 

In [10]:
happiness_transformed.to_sql(name = 'world_happiness', con = engine, if_exists = 'append', index = False)

153

In [11]:
freedom_transformed.to_sql(name = 'human_freedom_index', con = engine, if_exists = 'append', index = False)

165